In this task you will develop a system to detect irony in text. We will use the data from the SemEval-2018 task on irony detection. You should use the file `SemEval2018-T3-train-taskA.txt` from Blackboard it consists of examples as follows:

```csv
Tweet index     Label   Tweet text
1       1       Sweet United Nations video. Just in time for Christmas. #imagine #NoReligion  http://t.co/fej2v3OUBR
2       1       @mrdahl87 We are rumored to have talked to Erv's agent... and the Angels asked about Ed Escobar... that's hardly nothing    ;)
3       1       Hey there! Nice to see you Minnesota/ND Winter Weather 
4       0       3 episodes left I'm dying over here
```


**Introduction**

In this assignment I have constructed code blocks which have been built based on the file SemVal2018. In the file there is tweets and sentiment label written based on which my model will be able to predict whether the text is ironic and non-ironic seeing non seen data. 
I this assignment I have made a model of types in which at the first part I have built a clasiical model based on bag-of-words strategy and naive bayes algorithm based classificasion model.
The secod model is based on Deep learning and Recurrent neural network which is based on train data test data.
In the third part I have proposed a different model which is giving me better accuracy thant the 4th model.

All the codes are written in python and the code blocks are being given with comments to better understand.

In [2]:
#This code block has been written to import the files in to the google drive collab session to work further.
from google.colab import files
uploaded = files.upload()
from nltk.tokenize import sent_tokenize

Saving SemEval2018-T3-train-taskA.txt to SemEval2018-T3-train-taskA.txt


In [3]:
#This part has been written to import the nltk liabrary.
import nltk
#nltk.download('punkt')#Please uncomment this part if the plunck is not there in cloud drive session.
#nltk.download('averaged_perceptron_tagger') #Please uncomment this part if the plunck is not there in cloud drive session.

#This part is to read the file.
inputfile='SemEval2018-T3-train-taskA.txt' #Location of the file
f = open(inputfile,'r')
document = f.readlines()[1:]
f.close()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:
#I have written this code block beacuse the file has been imported and opened as text file in read mode. Now the text file is tab seperated which was comming
#in wrong format in dataframe. To correct the format so that the columns will be written in right way, I have made the csv from which will be taking tab 
#seperated data and I have worked on this dataset.
import csv
import itertools
with open('SemEval2018-T3-train-taskA.txt', 'r') as infile, open('SemEval2018-T3-train-taskA.CSV', 'w') as outfile:
        stripped = (line.strip() for line in infile)
        lines = (line.split("\t") for line in stripped if line)
        writer = csv.writer(outfile)
        writer.writerows(lines)

In [0]:
#This part is to take the data set into a data frame to work further.
import pandas as pd
df = pd.read_csv("SemEval2018-T3-train-taskA.CSV", sep=",")
df
#df = pd.Dataframe(list_of_lists)

# Task 1 (5 Marks)

Read all the data and find the size of vocabulary of the dataset (ignoring case) and the number of positive and negative examples.

In [6]:
#This code has been written to get the total number of tweets words in the text file. There are positive and negative data which is basically can be identified
#by the lavel 1 and 0. The first function which I have written will be capable to return the total word sets of the tweet text which is there in the dataframe.
#The next code has been written which is written to get the total number of word vocabulary in the file as well as how many number of positive and negative
#texts are there.
def word_vocab(document):
  data = document['Tweet text']
  words = [nltk.word_tokenize(sent.lower()) for sent in data] 
  wordsList = [item.strip() for sublist in words for item in sublist]
  return wordsList

def ie_preprocess(document):
    #code goes here
  words = word_vocab(document)
  a=[]
  positiveCount =0
  negativeCount =0
  totalWordList = len(set(words)) 
  print("Total number of vocabulary number is: ", totalWordList)#Total number in vocabulary
  dataTypes = document['Label']
  for i in range(len(dataTypes)):
    if(dataTypes[i] == 1):
      positiveCount = positiveCount + 1 
    else :
      negativeCount = negativeCount + 1
  negative_text_count = negativeCount
  positive_text_count = positiveCount
  print("Number of Positive examples is :", positive_text_count)
  print("Number of Negative examples is :", negative_text_count)
  #return words
ie_preprocess(df)

Total number of vocabulary number is:  13442
Number of Positive examples is : 1911
Number of Negative examples is : 1923


# Task 2 (20 Marks)

Develop a classifier using the Naive Bayes model to predict if an example is ironic. The model should convert each Tweet into a bag-of-words and calculate

$p(\text{Ironic}|w_1,\ldots,w_n) \propto \prod_{i=1,\ldots,n} p(w_i \in \text{tweet}| \text{Ironic}) p(\text{Ironic})$

$p(\text{NotIronic}|w_1,\ldots,w_n) \propto \prod_{i=1,\ldots,n} p(w_i \in \text{tweet}| \text{NotIronic}) p(\text{NotIronic})$

You should use add-alpha smoothing to calculate probabilities

In [0]:
#This code has been written to build a classic classifier model with the help of Naive bayes which is very good way to build a classifier for sentiment 
#analysis.
#In the first funtion I have written a prediction function which is capable of analysing the text data of test set and build on the vocabulary and the frequency
#based on the training dataset by the model, I can predict whether the text is ironic and non-ironic. I have used add-one smoothing to predict the probabilirity
#to restrict the out-of-vocabulary problem.
#The second function is a small function which is basically to use the counter to make the frequency and train data set bag-of-words set.
#The third function is to build the model based on the training dataset. The dataset has been build based on bag-of-words model and returned the probability of 
#probability of negative class and positive class, number of negative words and positive words, negative sentence and positive sentence. 
from collections import Counter
import re

dictionary = {}

def predictClassWord(text, counts, class_prob, class_count):
  prediction = 1
  sentence = text[2]
  text_counts = Counter(sentence)#re.split(" ", sentence))
  for word in text_counts:
      #print(word)
      prediction *=  text_counts.get(word) * ((counts.get(word,0) + 1) / (sum(counts.values()) + class_count))
      #print(text_counts.get(word))
  return prediction * class_prob


def counter_for_text(text):
  return Counter(text)

def prediction(document,train):#X,y):
    #code goes here
  a=[]
  positiveTextWords=[]
  negativeTextWords = []
  positiveCount =0
  negativeCount =0
  for i in range(len(train)):
    if(train[i][1] == 1):
      positiveCount = positiveCount + 1 
      positiveTextWords.append(train[i][2])
    else :
      negativeCount = negativeCount + 1
      negativeTextWords.append(train[i][2])
  negative_text_count = negativeCount
  positive_text_count = positiveCount
  
  
  wordsListPositive = [item.strip() for sublist in positiveTextWords for item in sublist]
  wordsListNegative = [item.strip() for sublist in negativeTextWords for item in sublist]
  
  #This code I have written to have the list of all the words
  ab=[]
  for i in range(len(train)):
    ab.append(train[i][2])
  wordsList = [item.strip() for sublist in ab for item in sublist]
  
  
  prob_positive = positive_text_count / len(train)#totalWordList
  prob_negative = negative_text_count / len(train)#totalWordList
  
  print(prob_positive)
  print(prob_negative)
  
  negative_counts = counter_for_text(wordsListNegative)
  positive_counts = counter_for_text(wordsListPositive)
  #This is needed for smoothing
  negativeWordCount = len(set(wordsListNegative))
  positiveWordCount = len(set(wordsListPositive))
  print(negativeWordCount)
  print(positiveWordCount)
  
  return negative_counts,positive_counts,prob_negative,prob_positive,negativeWordCount,positiveWordCount#negative_text_count,positive_text_count
#prediction(df)

# Task 3 (15 Marks)

Divide the data into a training and test set and justify your split.

Choose a suitable evaluation metric and implement it. Explain why you chose this evaluation metric.

Evaluate the method in Task 2 according to this metric.

In [0]:
#The below code has been written to split the dataset in the said format which will be in tupple format. This training and test data will be used further also
#to use the data. I have taken the help of scikit learning which will be used to split the data into 66.666 and 33.333 for mat. I have chosen this split to
#give the model more train data so that it can take more words to be trained.
data_list = []

for i in range(0,len(df)):
  tup = (int(df.iloc[i,:]["Tweet index"]),
         int(df.iloc[i,:]["Label"]),
         nltk.word_tokenize(df.iloc[i,:]["Tweet text"].rstrip().lower())
        )
  data_list.append(tup)
from sklearn.model_selection import train_test_split 
train,test = train_test_split(data_list, test_size=0.33, random_state=1)

In [0]:
#This function has been wirtten to give the classic classifier model test data based on which the model will predict whether the tweet is ironic or non ironic
def outputDecision(text,negative_counts,prob_negative,negative_text_count,positive_counts,prob_positive,positive_text_count):
  a=[]
  negativePrediction = predictClassWord(text, negative_counts, prob_negative, negative_text_count)
  positivePrediction = predictClassWord(text, positive_counts, prob_positive, positive_text_count)
  
  if(negativePrediction > positivePrediction):
    a.append(1)
    return 0
  return 1

In [10]:
#This is to call the train function to train the model
trainData = prediction(df,train)#X_train,y_train) 

0.49961059190031154
0.5003894080996885
6263
5529


In [0]:
#The function has been written to call the prediction function with the test data.
predictions = [outputDecision(r,trainData[0],trainData[2],trainData[4],trainData[1],trainData[3],trainData[5]) for r in test]

In [12]:
#This code I have written to predict the accuracy of the system. I have compared the original label of the test data with the predicted label. 
#I have multiplied it with the 100 to get the accuracy in percentage.
#I have taken the validation matix like confusion matrix, classification report and area under curver. 
#I have chosen this evalution process to print what is true values are coming from the classification. The AUC curve is giving that my system isgiving area
#between 1 to 0 which is 63.
#The confusion matrix is printing how many true positive and true negative the classifier has able to identify.
#I have also printed the precision and recall and depending on that F-Score so that I can say that based on my true value how many retrieved value are gold
#standard value.
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import accuracy_score
actual=[]
for i in test:
  actual.append(i[1])
count = 0
for i in range(len(predictions)):
  if(predictions[i] == actual[i]):
    count = count + 1
print("The accuracy is: -", count/len(predictions)*100)

# Generate the roc curve using scikits-learn.
fpr, tpr, thresholds = metrics.roc_curve(actual, predictions, pos_label=1)

cm = confusion_matrix(actual, predictions)  
cr = classification_report(actual, predictions)
acc = accuracy_score(actual, predictions)

# Measure the area under the curve.  The closer to 1, the "better" the predictions.
print("AUC of the predictions: {0}".format(metrics.auc(fpr, tpr)))

print("The report for the same is below: -")
print(cr)
print("The confusion matrix for the same is below: -")
print(cm)

The accuracy is: - 63.03317535545023
AUC of the predictions: 0.6317438052832298
The report for the same is below: -
              precision    recall  f1-score   support

           0       0.71      0.45      0.55       638
           1       0.59      0.81      0.69       628

   micro avg       0.63      0.63      0.63      1266
   macro avg       0.65      0.63      0.62      1266
weighted avg       0.65      0.63      0.62      1266

The confusion matrix for the same is below: -
[[289 349]
 [119 509]]


# Task 4 (20 Marks)

Run the following code to generate a model from your training set. The training set should be in a variable  called `train` and is assumed to be of the form:

```
[(1, 1, ['sweet', 'united', 'nations', 'video', '.', 'just', 'in', 'time', 'for', 'christmas', '.', '#', 'imagine', '#', 'noreligion', 'http', ':', '//t.co/fej2v3oubr']), 
 (2, 1, ['@', 'mrdahl87', 'we', 'are', 'rumored', 'to', 'have', 'talked', 'to', 'erv', "'s", 'agent', '...', 'and', 'the', 'angels', 'asked', 'about', 'ed', 'escobar', '...', 'that', "'s", 'hardly', 'nothing', ';', ')']), 
 (3, 1, ['hey', 'there', '!', 'nice', 'to', 'see', 'you', 'minnesota/nd', 'winter', 'weather']), 
 (4, 0, ['3', 'episodes', 'left', 'i', "'m", 'dying', 'over', 'here']), 
 ...
]
 ```



In [13]:
#The below code has been written to apply the keras framework to implement the RNN sequence model and stacked LSTM to make the classifier.
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np

## These values are taking from task 3 which I have already created. I have also commented the function of train test split. If it was not created
#this function can be called to make the split data.
train, test =train,test#train_test_split(data_list, test_size=0.33, random_state=1) # #task3()

def make_dictionary(train, test):
    dictionary = {}
    for d in train+test:
        for w in d[2]:
            if w not in dictionary:
                dictionary[w] = len(dictionary)
    return dictionary

class KerasBatchGenerator(object):
    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        self.current_idx = 0
        self.current_sent = 0
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, 2))
        while True:
            for i in range(self.batch_size):
                # Choose a sentence and position with at lest num_steps more words
                while self.current_idx + self.num_steps >= len(self.data[self.current_sent][2]):
                    self.current_idx = self.current_idx % len(self.data[self.current_sent][2])
                    self.current_sent += 1
                    if self.current_sent >= len(self.data):
                        self.current_sent = 0
                # The rows of x are set to values like [1,2,3,4,5]
                x[i, :] = [self.vocabulary[w] for w in self.data[self.current_sent][2][self.current_idx:self.current_idx + self.num_steps]]
                # The rows of y are set to values like [[1,0],[1,0],[1,0],[1,0],[1,0]]
                y[i, :, :] = [[self.data[self.current_sent][1], 1-self.data[self.current_sent][1]]] * self.num_steps
                self.current_idx += self.skip_step
            yield x, y

# Hyperparameters for model
vocabulary = make_dictionary(train, test)
num_steps = 5
batch_size = 20
num_epochs = 50 # Reduce this if the model is taking too long to train (or increase for performance)
hidden_size = 90 # Increase this to improve perfomance (or increase for performance)
use_dropout=True

# Create batches for RNN
train_data_generator = KerasBatchGenerator(train, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(test, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)

# A double stacked LSTM with dropout and n hidden layers
model = Sequential()
model.add(Embedding(len(vocabulary), hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(2)))
model.add(Activation('softmax'))

# Set optimizer and build model
optimizer = Adam()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

# Train the model
model.fit_generator(train_data_generator.generate(), len(train)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(test)//(batch_size*num_steps))

# Save the model
model.save("final_model.hdf5")

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/50
25/25 [==============================] - 5s 189ms/step - loss: 0.6862 - categorical_accuracy: 0.5964 - val_loss: 0.6771 - val_categorical_accuracy: 0.5750
Epoch 2/50
25/25 [==============================] - 1s 38ms/step - loss: 0.6781 - categorical_accuracy: 0.5616 - val_loss: 0.6791 - val_categorical_accuracy: 0.5500
Epoch 3/50
25/25 [==============================] - 1s 38ms/step - loss: 0.6810 - categorical_accuracy: 0.5588 - val_loss: 0.6731 - val_categorical_accuracy: 0.5517
Epoch 4/50
25/25 [==============================] - 1s 37ms/step - loss: 0.6785 - categorical_accuracy: 0.5736 - val_loss: 0.6801 - val_categorical_accuracy: 0.5142
Epoch 5/50
25/25 [========

Now consider the following code:

In [14]:
model = load_model("final_model.hdf5")

x = np.zeros((1,num_steps))
x[0,:] = [vocabulary["this"],vocabulary["the"],vocabulary["an"],vocabulary["easy"],vocabulary["test"]]
# print(model.predict(x))
x.shape

(1, 5)

In [0]:
#In this part I have written the code which is predicting the text accuracy based on the tweets words. This part I have done with Subhajit and I have written down
#which part has done by whome. We have taken the approach of sentence word probability multiplication. As our classifier is able to predict words proabability o
#on 5 words based, everytime we have divided the sentence into 5 words token and sent 5 words into vocabulary. Then we got the 5 words individual prediction
#of being ironic and non ironic and captured that into matrix.
#Next again we have sent the 5 words based on window feature which like first sending 12345 and then 23456. These are the index positions of the words.
#As we are getting the probability of words of being ironinc and no ironic multiple times, we have multiplied all the probabilities and got the probability of the sentence.

from collections import defaultdict
model = load_model("final_model.hdf5")

predict_label = []

for i in range(len(test)):

    line = test[i][2]

    dictPos = defaultdict(list)

    dictNeg = defaultdict(list)

 
    # Shubhajit (#18231026) : Start Code 
    for i in range(0,len(line)-4):

 

        str_ln = line[i:i+5]

        lin_seg = np.array([vocabulary[i] for i in line[i:i+5]])

        lin_seg = lin_seg.reshape(1,5)

        lin_seg_pred = model.predict(lin_seg)

        lin_seg_pos = lin_seg_pred.reshape(5,2)[:,0:1].flatten()

        lin_seg_neg = lin_seg_pred.reshape(5,2)[:,1:].flatten()

        d_pos = dict(zip(str_ln,lin_seg_pos))

        d_pos = {k: [v] for k, v in d_pos.items()}

        d_neg = dict(zip(str_ln,lin_seg_neg))

        d_neg = {k: [v] for k, v in d_neg.items()}

 

        for k,v in d_pos.items():

            dictPos[k].extend(v)

 

        for k,v in d_neg.items():

            dictNeg[k].extend(v)
    # Shubhajit (#18231026) :End Code
    
    # Koustava (#18234857) : Start Code
    multiplyPositive=1

    multiplyNegative=1

    for k,v in dictPos.items():

        for j in v:

            multiplyPositive = multiplyPositive * j

    for k1,v1 in dictNeg.items():

        for i in v1:

            multiplyNegative = multiplyNegative * i

    if(multiplyPositive>multiplyNegative):

        predict_label.append(1)

    else:

        predict_label.append(0)
    
    # Koustava (#18234857) : End Code

In [16]:
#This code I have written to predict the accuracy of the system. I have compared the original label of the test data with the predicted label. 
#I have multiplied it with the 100 to get the accuracy in percentage.
#I have taken the validation matix like confusion matrix, classification report and area under curver. 
#I have chosen this evalution process to print what is true values are coming from the classification. The AUC curve is giving that my system isgiving area
#between 1 to 0 which is 63.
#The confusion matrix is printing how many true positive and true negative the classifier has able to identify.
#I have also printed the precision and recall and depending on that F-Score so that I can say that based on my true value how many retrieved value are gold
#standard value.
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn.metrics import accuracy_score
#actual = [int(r) for r in y_test]
actual=[]
for i in test:
  actual.append(i[1])
count = 0
for i in range(len(actual)):
  if(predict_label[i] == actual[i]):
    count = count + 1
print(count/len(predict_label)*100)
# Generate the roc curve using scikits-learn.
fpr, tpr, thresholds = metrics.roc_curve(actual, predictions, pos_label=1)

cm = confusion_matrix(actual, predictions)  
cr = classification_report(actual, predictions)
acc = accuracy_score(actual, predictions)

# Measure the area under the curve.  The closer to 1, the "better" the predictions.
print("AUC of the predictions: {0}".format(metrics.auc(fpr, tpr)))

print("The report for the same is below: -")
print(cr)
print("The confusion matrix for the same is below: -")
print(cm)

57.345971563981045
AUC of the predictions: 0.6317438052832298
The report for the same is below: -
              precision    recall  f1-score   support

           0       0.71      0.45      0.55       638
           1       0.59      0.81      0.69       628

   micro avg       0.63      0.63      0.63      1266
   macro avg       0.65      0.63      0.62      1266
weighted avg       0.65      0.63      0.62      1266

The confusion matrix for the same is below: -
[[289 349]
 [119 509]]


Using the code above write a function that can predict the label using the LSTM model above and compare it with the evaluation performed in Task 3

# Task 5 (40 Marks)

Suggest an improvement to either the system developed in Task 2 or 4 and show that it improves according to your evaluation metric.

Please note this task is marked according to: demonstration of knowledge from the lecutures (10), originality and appropriateness of solution (10), completeness of description (10), technical correctness (5) and improvement in evaluation metric (5).

**Different Approcah**

In this part I have taken a different approach to build a classifier which will be able to train the model based on the training set vocabulary and will be able to predict the sentiments of the unseen tweets in the future test data.

**Model**

My classifier is based on the theory of CNN and LSTM. There are few works going on where CNN model is trying to be used. The CNN model is taking the data word vector similarly as image does and then trying to store the same using LSTM model. My model is first consistimg of CNN model where the tweets words will be passed and then it will be passed to a bidirectional LSTM model. I have used softmax activation function to get the probability of the words to identify the unseen tweets. I have also used categorical_crossentropy loss function which will be optimised.

**Process**

CNN is efficient analysis text as the convulution layerscan extract features horizontally from multiple words. The model is also very efficient where position of the words does not matter to identify the classification. On the other hand LSTM is very good in predicting the last changing sentiments of text to be memorised based on sequence. I have taken that approach to memorise what I have fitted into the model to check the sentiment of the texts tweets. I have made a LSTM model of hundred unites and also made it bidirectional. My bidirectional LSTM is now capable of reading and memorise the string from both end which will give a good model prediction. The LSTM model is then passed to a fully connected layer which I have applied using dense function of keras. 

**Prediction**

As my model is now ready to be classified, I have passes the test data set into the classifier and then got the prediction about the sentiment label for each tweet

**Evaluation**

I have used confusion matrix, accuracy score and precision recall data using scikit learning. I have choosen precision recall model to identify how my model is behaving on gold standard data and the prediction data. From the model I can see that my precision and recall is very nearer which means the prediction is at good side. 

My model is now giving the prediction 60%+ which has an improvement of previous RNN LSTM model

[Note:- I have made this model on very ground theory of CNN data acceptance format and one LSTM model. I have tried to incorporate the policy of fedding images in CNN and replaced it by text to see whether this is efficient.]

In [17]:
#This part is to just see the columns of the dataframe containing the tweeter file.
df.columns

Index(['Tweet index', 'Label', 'Tweet text'], dtype='object')

In [18]:
#Here I am making my X value which will then be fitted to scikit learning training test split model
X= df['Tweet text'].values
X.shape

(3834,)

In [19]:
#Here I am classiying the sentiment labels for the tweets
y = pd.get_dummies(df['Label']).values
y.shape

(3834, 2)

In [20]:
#I have taken the help of scikit learning liabrary to make training ans test split of 80 - 20 percent. I have made this split because our corpus is small
#and neural network needs more data to be learned the model. So I have increased the training size.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    stratify=y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3067,) (767,) (3067, 2) (767, 2)


In [0]:
#Initialization of the model importing liabraries and mentioning number of epochs,batch size and maximum features
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense,Embedding,Conv1D,MaxPooling1D,LSTM
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from keras.layers import Bidirectional

batch_size = 128
epochs = 20
max_features = 20000

In [0]:
#I have used standard keras tokenizer model to tokenize the training and test data tweet words which will be then feed to the model. I have used a max_features
#techniques in which I have passed the maximum features that is  ighest number of words could be in vocabulary of text.
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [0]:
#I have padded the sequence with the use of keras model. I have here padded till the maximum number of words in the vocabulary that is 33.
from keras.preprocessing import sequence
max_words = 33 #This is the number of maximum words in the vocabulary
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)
#print(X_train)

In [0]:
#This code has been written to make the model. I have used the architecture of CNN-LSTM model. 
#First we have embedded with the highest number of words in the vocabulary. Then I have passed this through CNN layer which has taken the shape as X_train data
#shape. Then I have passed it thorugh a LSTM layer. This LSTM layer has 100 cells in it, I have made it bidirectional to read train more accurately as it will
#learn from for ward and backward. 
#I have taken activation function as softmax to take the probability of the words of being ironic and non-ironic.
def CnnLstmModel(max_features, embed_dim):
    #np.random.seed(seed)
    K.clear_session()
    model = Sequential()
    model.add(Embedding(max_features, embed_dim, input_length=X_train.shape[1]))
    model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling1D(pool_size=2))   
    model.add(Bidirectional(LSTM(100, dropout=0.2,  recurrent_dropout=0.2)))
    #model.add(LSTM(100, dropout=0.2,  recurrent_dropout=0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    print(model.summary())
    return model

In [0]:
#This is the code which I have written to fit the train and test data as validation set in the model. My train will be x-train containing the data
#and y-train for the training data label. 
def model_train(model):
    # train the model
    model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                          epochs=epochs, batch_size=batch_size, verbose=2)
    # plot train history
    #plot_model_history(model_history)

In [26]:
# train the model
max_features = 13442 #This is the number of words in the vocabulary that we got in the first question. I have used that part.
embed_dim = 100
model = CnnLstmModel(max_features, embed_dim)
model_train(model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 33, 100)           1344200   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 33, 32)            9632      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 16, 32)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               106400    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 402       
Total params: 1,460,634
Trainable params: 1,460,634
Non-trainable params: 0
_________________________________________________________________
None
Train on 3067 samples, validate on 767 samples
Epoch 1/20
 - 6s - loss: 0.6901 - acc: 0.5383 - val_loss: 0.6843 - val_acc: 0.58

In [0]:
#This code I have written to predict the accuracy of the system. I have compared the original label of the test data with the predicted label. 
#I have multiplied it with the 100 to get the accuracy in percentage.
#I have taken the validation matix like confusion matrix, classification report and area under curver. 
#I have chosen this evalution process to print what is true values are coming from the classification. The AUC curve is giving that my system isgiving area
#between 1 to 0 which is 63.
#The confusion matrix is printing how many true positive and true negative the classifier has able to identify.
#I have also printed the precision and recall and depending on that F-Score so that I can say that based on my true value how many retrieved value are gold
#standard value.
import numpy as np
def evaluation(): 
    # predict class with test set
    y_pred_test =  model.predict_classes(X_test, batch_size=batch_size, verbose=0)
    print('Accuracy:\t{:0.1f}%'.format(accuracy_score(np.argmax(y_test,axis=1),y_pred_test)*100))
    
    #classification report
    print('\n')
    print(classification_report(np.argmax(y_test,axis=1), y_pred_test))

    #confusion matrix
    confmat = confusion_matrix(np.argmax(y_test,axis=1), y_pred_test)
    print("The Confusion matrix is: -")
    print(confmat)


In [28]:
#This code is written to call the function which will evalute the test data based on the system
evaluation()

Accuracy:	60.5%


              precision    recall  f1-score   support

           0       0.59      0.67      0.63       385
           1       0.62      0.54      0.58       382

   micro avg       0.60      0.60      0.60       767
   macro avg       0.61      0.60      0.60       767
weighted avg       0.61      0.60      0.60       767

The Confusion matrix is: -
[[257 128]
 [175 207]]


**Reference**

To get the idea of how keras code works on CNN-LSTM model I have taken the reference of the below website 

Reference:- https://www.kaggle.com/dundee2002/bitcoin-tweets-sentiment-analysis-glove-cnn-lstm